## Scraping shot selection for each player per year from 1996-2016.
- Selenium webdriver and bs4.BeautifulSoup were imported to scrape the data on nba stats.


In [35]:
import pandas as pd

import requests
import time
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
from selenium import webdriver
from bs4 import BeautifulSoup
import json

chromedriver = "/Users/amos/Desktop/chromedriver"

In [36]:
# opens selenium webdriver
browser = webdriver.Chrome(executable_path= chromedriver)

In [37]:
browser.get('http://stats.nba.com/league/player/#!/shooting/')
#json html format
HTML_logs = browser.page_source
years = Selector(text=HTML_logs).xpath("//select[@name='Season']/option/text()").extract()

In [38]:
stats_url = 'http://stats.nba.com/stats/leaguedashplayershotlocations?College=&Conference=&Country=&DateFrom=&DateTo=\
&DistanceRange=5ft+Range&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=\
&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&PlayerExperience=\
&PlayerPosition=&PlusMinus=N&Rank=N&Season=%s\
&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='

In [39]:
# Find Header or column names
link = stats_url % '2001-02'
browser.get(link)
HTML = browser.page_source
soup = BeautifulSoup(HTML, 'lxml')
text = soup.findAll('pre')[0].getText().encode('utf-8')
json_text = json.loads(text)
shot_ranges = json_text['resultSets']['headers'][0]['columnNames']
new_ranges = [rang.replace('-','_').replace('.','').replace(' ', '').
                  replace('+','above').lower() for rang in shot_ranges]
columns = json_text['resultSets']['headers'][1]['columnNames']
fg_list = ['FGM', 'FGA', 'FG_PCT']
headers = columns[:5]
for rang in new_ranges:
    for f in fg_list:
        headers.append(rang + '_'+ f)

In [40]:
# Create Data frame of each player's shot selection for each year from 1996-2016 and save to csv file.
for year in years:
    print year
    link = stats_url % year
    #selenium browser opens link
    browser.get(link)
    HTML = browser.page_source
    soup = BeautifulSoup(HTML, "lxml")
    get_text = soup.findAll('pre')[0].getText().encode('utf-8')
    json_text = json.loads(get_text)

    
    df = pd.DataFrame(json_text['resultSets']['rowSet'], columns=headers) 
    save_csv = '/Users/amos/Desktop/DSI-SF-2-amoskim25/copy_capstone/datasets/shot_range/shot_range%s.csv' % year
    df.to_csv(save_csv)
#     time.sleep(5*np.random.rand())
 

2015-16
2014-15
2013-14
2012-13
2011-12
2010-11
2009-10
2008-09
2007-08
2006-07
2005-06
2004-05
2003-04
2002-03
2001-02
2000-01
1999-00
1998-99
1997-98
1996-97


In [41]:
# testing out shot selection data for 2001 
stats_2001 = pd.read_csv('/Users/amos/Desktop/DSI-SF-2-amoskim25/copy_capstone/datasets/shot_range/shot_range2001-02.csv')
print "Shot selection based on distance"
print "Dimension: ", stats_2001.shape
print "Column names: ", stats_2001.columns

Shot selection based on distance
Dimension:  (440, 33)
Column names:  Index([u'Unnamed: 0', u'PLAYER_ID', u'PLAYER_NAME', u'TEAM_ID',
       u'TEAM_ABBREVIATION', u'AGE', u'lessthan5ft_FGM', u'lessthan5ft_FGA',
       u'lessthan5ft_FG_PCT', u'5_9ft_FGM', u'5_9ft_FGA', u'5_9ft_FG_PCT',
       u'10_14ft_FGM', u'10_14ft_FGA', u'10_14ft_FG_PCT', u'15_19ft_FGM',
       u'15_19ft_FGA', u'15_19ft_FG_PCT', u'20_24ft_FGM', u'20_24ft_FGA',
       u'20_24ft_FG_PCT', u'25_29ft_FGM', u'25_29ft_FGA', u'25_29ft_FG_PCT',
       u'30_34ft_FGM', u'30_34ft_FGA', u'30_34ft_FG_PCT', u'35_39ft_FGM',
       u'35_39ft_FGA', u'35_39ft_FG_PCT', u'40aboveft_FGM', u'40aboveft_FGA',
       u'40aboveft_FG_PCT'],
      dtype='object')
